In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col

In [2]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Kafkaa") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .getOrCreate()

In [3]:

# Đọc dữ liệu từ Kafka topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "192.168.1.197:9092") \
  .option("subscribe", "spark.public.adaniports") \
  .option("startingOffsets", "earliest") \
  .load()



In [4]:
from pyspark.sql.types import *

schema = StructType([
  StructField("Date", StringType(), True),
  StructField("symbol", StringType(), True),
  StructField("serials", StringType(), True),
  StructField("Prev Close", StringType(), True),
  StructField("Open", StringType(), True),
  StructField("high", StringType(), True),
  StructField("low", StringType(), True),
  StructField("Last", StringType(), True),
  StructField("Close", StringType(), True),
  StructField("vwap", StringType(), True),
  StructField("volume", StringType(), True),
  StructField("turnover", StringType(), True),
  StructField("trades", StringType(), True),
  StructField("Deliverable Volume", StringType(), True),
  StructField("%Deliverble", StringType(), True),
])

In [5]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Chuyển đổi dữ liệu từ binary sang string
df = df.selectExpr("CAST(value AS STRING)")

# Chuyển đổi dữ liệu từ JSON sang PySpark DataFrame
#schema = 'your_schema' # định nghĩa schema cho dữ liệu
df = df.select(from_json(col("value"), schema).alias("data"))

# Áp dụng các phép biến đổi và tính toán trên dữ liệu
result_df = df.select("data.*")

# Xuất kết quả
query = result_df \
    .writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("header", "true") \
    .option("checkpointLocation", "work") \
    .option("path", "work/da.csv") \
    .start()

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above ex

KeyboardInterrupt: 

In [ ]:
df_pyspark.filter("Salary<=20000").show()

In [ ]:
df_pyspark.groupBy('Departments').mean().show()

In [ ]:
df_pyspark.na.drop().show()

In [ ]:
df_pyspark.describe().show()